### 模型学习新知识遗忘旧知识

### 核心原因：共享参数权重
- 所有的知识都存储在大模型的参数中，当模型学习A知识的时候，将参数调整为“最适合”该知识的状态，当模型学习B知识的时候，调整的是同一组参数，使参数朝着B知识最优的方向进行改动，过程中不可避免地就影响了之前任务A调整好的参数状态

### 相同领域，不同子模块
- 模型在第一个模块的数据上训练后，在第二个模块的数据上训练，忘记了第一个模块的知识。

### 不同领域
- 模型从原来的领域迁移到另一个领域，在训练过程中忘了之前领域的知识。

### 解决方法
- 混合数据训练
- 模型checkpoint融合策略
- 正则化约束：弹性权重巩固（EWC）算法
  - EWC思想：一个模型在学习旧任务（任务A）时，其内部的参数（权重）对任务A的贡献度是不同的。有些参数是完成任务A的“关键支柱”，而另一些则不那么重要。因此，在学习新任务（任务B）时，不应该“一视同仁”地更新所有参数，而应该重点保护那些对旧任务至关重要的参数，让它们尽可能保持不变，同时允许那些次要的参数自由更新以适应新任务。
  - 费雪信息矩阵：衡量了模型参数的微小变动对模型输出结果影响的剧烈程度。
  - $L(\theta)=L_B(\theta)+\frac{\lambda}{2}\sum_iF_i(\theta_i-\theta_{A,i}^*)^2$
  - * **$L_B(\theta)$**: 这是**新任务B的标准损失函数**。模型需要最小化它来学习新知识。
    * **$\frac{\lambda}{2} \sum_i F_i (\theta_i - \theta_{A,i}^*)^2$**: 这就是**EWC的核心惩罚项**，它像一个“守护者”，保护旧知识。
    * **$\lambda$ (lambda)**：一个超参数，用来调节“守护者”的强度。$\lambda$ 越大，对旧知识的保护就越强，但学习新知识的能力可能会被削弱。
    * **$F_i$**：第 $i$ 个参数对于**旧任务A的重要性**（即费雪信息值）。
    * **$\theta_i$**：第 $i$ 个参数在学习任务B时的**当前值**。
    * **$\theta_{A,i}^*$**：第 $i$ 个参数在**完成任务A训练后的最优值**。这是我们要努力维持的“锚点”。
- 分层学习率 ：平常都是以相同学习率来调整所有的层
  - 对于模型的底层（输入端）更多是学习的是通用的表达方法，例如语法结构、词性关系等，这些几乎都适用于所有下游任务
  - 对于模型的高层（输出端）更多学习的是更加抽象的语义特征，使其更加适配下游任务
  - 所以，可以对底层使用更低的学习率，对高层使用更高的学习率
- 动态评估机制
  - 可以对中间的checkpoint进行测试，查看模型的微调效果，如果太过偏差，就需要增强原始的数据采样